In [25]:
import openai
import os

from pprint import pprint

from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

# Models and Prompt Templates

Adapted from Lesson 2 of [LangChain for LLM Application Development](https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/) by deeplearning.ai.

In [2]:
# Set up model
chat = ChatOpenAI(temperature=0.0)

# Set up prompt template
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

In [11]:
# Why is it messages plural?
prompt_template.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True), additional_kwargs={})

In [10]:
# Need to pass these each time we use the template - much nicer and more modular!
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [12]:
output_style = """British English in a calm, friendly, and respectful tone"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# Create the prompt using the template
customer_messages = prompt_template.format_messages(
                    style=output_style,
                    text=customer_email)

customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is British English in a calm, friendly, and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, example=False)]

In [15]:
customer_response = chat(customer_messages)
# Now translated into British English
customer_response.content

"Arrr, I must say I'm quite frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to add to my troubles, the warranty doesn't cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this moment, my friend!"

In [18]:
# Let's translate the service reply back into the original language (Pirate English)

service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

# This name is a bit odd. We aren't formatting messages, we are formatting the prompt
service_messages = prompt_template.format_messages(
                    style=service_style_pirate,
                    text=service_reply)

service_messages[0].content

"Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"

In [21]:
# Now the service response (originally in English) is in the user's language (Pirate English)
service_response = chat(service_messages)
service_response.content

"Ahoy there, matey! I regret to inform ye that the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to secure the lid afore startin' it. Aye, tough luck, me heartie! Fare thee well!"

# Output Parsers

## The Slow, Brittle Way

In [22]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# Define our own template
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

# Create prompt template from our template string
prompt_template = ChatPromptTemplate.from_template(review_template)
# Fill in prompt template to create messages
messages = prompt_template.format_messages(text=customer_review)
# Pass messages to chat and get response
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
# This is brittle because the response is a string and still requires further processing
print(response.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


## Using Output Parsers

In [27]:
# Define each part of the response as a schema
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

# Create output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# Automatically (!) create format instructions for the model output
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [40]:
# This seems a bit excessive length wise
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

# Much shorter and gets the job done.
review_template_3 = """
For the following text, extract the following information and format it like this:

{format_instructions}

text: {text}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_3)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)
response = chat(messages)
print('Original response content: ', response.content)
parsed_response = output_parser.parse(response.content)
print('Parsed response: ', parsed_response)
print()
print('!!! Parsed response is type: ', type(parsed_response), ' !!!')
print()
print('parsed_response.get("gift"): ', parsed_response.get("gift"))

Original response content:  ```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "slightly more expensive than the other leaf blowers out there"
}
```
Parsed response:  {'gift': False, 'delivery_days': '2', 'price_value': 'slightly more expensive than the other leaf blowers out there'}

!!! Parsed response is type:  <class 'dict'>  !!!

parsed_response.get("gift"):  False


# Memory

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationSummaryBufferMemory
)

In [43]:
# Model + memory = conversation
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True # shows full prompt for each prediction
)

In [45]:
conversation.predict(input="Hello, it's really sunny today!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, it's really sunny today!
AI:

> Finished chain.


"Hello! Yes, it is indeed sunny today. The temperature is around 25 degrees Celsius, and there are only a few scattered clouds in the sky. It's a perfect day to go outside and enjoy the sunshine!"

In [47]:
conversation.predict(input="Is it so hot that I need to wear suncream?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, it's really sunny today!
AI: Hello! Yes, it is indeed sunny today. The temperature is around 25 degrees Celsius, and there are only a few scattered clouds in the sky. It's a perfect day to go outside and enjoy the sunshine!
Human: Is it so hot that I need to wear suncream?
AI:

> Finished chain.


"The temperature is 25 degrees Celsius, which is not extremely hot. However, it's always a good idea to wear sunscreen when you're going to be exposed to the sun for an extended period of time, regardless of the temperature. Sunscreen helps protect your skin from harmful UV rays, which can cause sunburn and increase the risk of skin cancer. So, it's better to be safe and apply some sunscreen before heading out."

In [48]:
print(memory.buffer)

Human: Hello, it's really sunny today!
AI: Hello! Yes, it is indeed sunny today. The temperature is around 25 degrees Celsius, and there are only a few scattered clouds in the sky. It's a perfect day to go outside and enjoy the sunshine!
Human: Is it so hot that I need to wear suncream?
AI: The temperature is 25 degrees Celsius, which is not extremely hot. However, it's always a good idea to wear sunscreen when you're going to be exposed to the sun for an extended period of time, regardless of the temperature. Sunscreen helps protect your skin from harmful UV rays, which can cause sunburn and increase the risk of skin cancer. So, it's better to be safe and apply some sunscreen before heading out.


In [49]:
memory.load_memory_variables({})

{'history': "Human: Hello, it's really sunny today!\nAI: Hello! Yes, it is indeed sunny today. The temperature is around 25 degrees Celsius, and there are only a few scattered clouds in the sky. It's a perfect day to go outside and enjoy the sunshine!\nHuman: Is it so hot that I need to wear suncream?\nAI: The temperature is 25 degrees Celsius, which is not extremely hot. However, it's always a good idea to wear sunscreen when you're going to be exposed to the sun for an extended period of time, regardless of the temperature. Sunscreen helps protect your skin from harmful UV rays, which can cause sunburn and increase the risk of skin cancer. So, it's better to be safe and apply some sunscreen before heading out."}

In [50]:
memory = ConversationBufferMemory()
memory.save_context({'input': 'Yo'}, {'output': 'Sup dawg'})
memory.buffer

'Human: Yo\nAI: Sup dawg'

In [51]:
memory.load_memory_variables({})

{'history': 'Human: Yo\nAI: Sup dawg'}

In [52]:
memory.save_context({'input': 'Nice trainers'}, {'output': 'Thanks bro, they were expensive'})
memory.buffer

'Human: Yo\nAI: Sup dawg\nHuman: Nice trainers\nAI: Thanks bro, they were expensive'

## ConversationBufferWindowMemory

In [56]:
# Just remember the last message
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

conversation.predict(input='my favourite food is pizza')

"Oh, pizza! That's a popular choice for many people. There are so many different types of pizza toppings and crusts to choose from. Do you have a specific favorite type of pizza?"

In [59]:
conversation.predict(input='what is the capital of Germany?')

'The capital of Germany is Berlin.'

In [60]:
# Doesn't remember as we set k=1
conversation.predict(input='what is my favourite food?')

"I'm sorry, but I don't have access to personal information about individuals, so I don't know what your favorite food is."

In [63]:
# This loads the variables actually in memory that the model uses
memory.load_memory_variables({})

# memory.buffer stores the whole convo and isn't necessarily what the model has access to at any given time

{'history': "Human: what is my favourite food?\nAI: I'm sorry, but I don't have access to personal information about individuals, so I don't know what your favorite food is."}

## ConversationTokenBufferMemory

Same thing but limit is on the number of tokens.

## ConversationSummaryMemory

This is really cool!

In [65]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging and eating my breakfast"},
                    {"output": "Coolio"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are eating breakfast. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [66]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human mentions that they are eating breakfast. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


"A good demo to show during the lunch meeting would be the latest version of the Language Learning Model (LLM) developed by our team. The LLM is an advanced AI model that can understand and generate human-like text in multiple languages. It has been trained on a vast amount of data and can perform tasks such as language translation, text summarization, and even creative writing. By showcasing the LLM's capabilities, we can demonstrate the power and potential of AI in various applications."

In [67]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings. The human mentions that they are eating breakfast. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the Language Learning Model (LLM), an advanced AI model that can understand and generate human-like text in multiple languages. The LLM has been trained on a vast amount of data and can perform tasks such as language translation, text summarization, and creative writing. By showcasing the LLM's capabilities, the AI believes they can demonstrate the power and potential of AI in various applications."}